In [1]:
from google.colab import drive
drive.mount('/content/drive/')
#change this path to the shared folder dir
# root_folder0="/content/drive/My Drive/Trading"
root_folder0="/content/drive/MyDrive/Data_Mining_Final_Project"
DB_path_basic="Db/DataSet_basic_all_tickers.csv"
DB_path="Db/DataSet_all_tickers.csv"
%cd $root_folder0
!pwd
!ls
#flat to decide if isntalll or not the libraries
install_libraries=True


KeyboardInterrupt: ignored

In [ ]:
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import date

In [ ]:
!pip install statsmodels

In [ ]:
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [ ]:
num_years=5

Tickers_DF= pd.read_csv(DB_path, sep='\t',header=[0,1], index_col=0)
print("DataSet Loaded")

In [ ]:
Tickers_DF.head(100)

In [ ]:
companyList = ["ATVI", "ADBE","AMD","ALXN","ALGN","GOOGL","GOOG","AMZN","AEP","AMGN","ADI","ANSS","AAPL","AMAT","ASML","TEAM","ADSK","ADP","BIDU","BIIB","BKNG","AVGO","CDNS","CDW","CERN","CHTR","CHKP","CTAS","CSCO","CTSH","CMCSA","CPRT","COST","CSX","DXCM","DOCU","DLTR","EBAY","EA","EXC","FB","FAST","FISV","FOXA","FOX","GILD","IDXX","ILMN","INCY","INTC","INTU","ISRG","JD","KDP","KLAC","KHC","LRCX","LULU","MAR","MRVL","MTCH","MXIM","MELI","MCHP","MU","MSFT","MRNA","MDLZ","MNST","NTES","NFLX","NVDA","NXPI","ORLY","OKTA","PCAR","PAYX","PYPL","PTON","PEP","PDD","QCOM","REGN","ROST","SGEN","SIRI","SWKS","SPLK","SBUX","SNPS","TMUS","TSLA","TXN","TCOM","VRSN","VRSK","VRTX","WBA","WDAY","XEL","XLNX","ZM"]
new_df = pd.DataFrame()

In [ ]:
for company in companyList:
  try:
    new_df[company] = Tickers_DF[company,"close"]
  except KeyError:
    print(company, " not present in the dataset")

In [ ]:
new_df

In [ ]:
# new_df.count(axis='columns')
step_size = 500
batch_size = 500
batch_df = []

for i in range(0, len(new_df.axes[0]), step_size):
  batch_df.append(new_df[i:i+100])


In [ ]:

## Plot
# fig, axes = plt.subplots(nrows=4, ncols=2, dpi=120, figsize=(10,6))
# plot_count = 0
# for i, ax in enumerate(axes.flatten()):
#     data = new_df[new_df.columns[i]]
#     ax.plot(data, color='red', linewidth=1)
#     # Decorations
#     ax.set_title(new_df.columns[i])
#     ax.xaxis.set_ticks_position('none')
#     ax.yaxis.set_ticks_position('none')
#     ax.spines["top"].set_alpha(0)
#     ax.tick_params(labelsize=6)
#     if plot_count is 2:
#       break
#     plot_count += 1

# plt.tight_layout();

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

maxlag=12
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df


In [ ]:
batch_cuasation_matrix = []
i = 0
for df in batch_df:
  print("Getting batch_causation_matrix of batch:", i)
  batch_cuasation_matrix.append(grangers_causation_matrix(df, variables = df.columns, verbose=False))  
  i += 1

In [ ]:
final_df = pd.concat([batch_cuasation_matrix[0], batch_cuasation_matrix[1], batch_cuasation_matrix[2]]).groupby(level=0).mean()

In [ ]:
final_df.to_csv('mean_causation_matrix.csv')